## Import Packages

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras import optimizers

In [2]:
num_samples = 20
num_epochs  = 10

tf.random.set_seed(0)
x = tf.random.normal((num_samples, 2), mean=0, stddev=1)
y = tf.reshape(3.145 * x[:, 0] +  6.323 * x[:, 1], (-1, 1))

square_loss = lambda: tf.reduce_mean(tf.pow(y - tf.matmul(x, w), 2))
evaluate = lambda i: print(''.join((
    f'epoch:{i+1:>3}{"":>4}weight 1: {w.numpy()[0, 0]:>7.3f}',
    f'{"":>4}weight 2: {w.numpy()[1, 0]:>7.3f}{"":>4}loss: {square_loss().numpy():>7.3f}'
)))

def train(num_epochs, optimizer, loss, weight):
    evaluate(-1)
    for i in range(0, num_epochs):
        optimizer.minimize(loss, var_list=[w])
        evaluate(i)

## Adaptive Gradient Descent (AdaGrad)
* Equation
$$
\begin{align}
\mathbf{w}^{(t+1)}&=\mathbf{w}^{(t)}-\frac{\eta}{\sqrt{\sum\limits_{i=1}^t\mathbf{g}^{(t)T}\mathbf{g}^{(t)}}+\epsilon} \mathbf{g}^{(t)}\\
\mathbf{g}^{(t)}&=∇_\mathbf{w}L(\mathbf{w}^{(t)})\\
\end{align}
$$
* Parameters
    * $\epsilon$ is a constant that make sure the denominator is not zero (default in Tensorflow/Keras $1e-7$).
    * $\eta$ is the learning rate (default in Tensorflow/Keras $0.001$)

* Properties
    * The magnitude of the update is smaller in later iterations.
    * Converge even with large learning rate.
    * The update become very inefficient in later iterations.
    * Vunerable to local minimum.


In [3]:
eta = 1
w = tf.Variable(tf.zeros((2, 1)))
opt = optimizers.Adagrad(learning_rate=eta)
train(num_epochs, opt, square_loss, w)

epoch:  0    weight 1:   0.000    weight 2:   0.000    loss:  37.729
epoch:  1    weight 1:   0.999    weight 2:   0.999    loss:  23.809
epoch:  2    weight 1:   1.574    weight 2:   1.639    loss:  17.052
epoch:  3    weight 1:   1.973    weight 2:   2.126    loss:  12.899
epoch:  4    weight 1:   2.267    weight 2:   2.523    loss:  10.088
epoch:  5    weight 1:   2.492    weight 2:   2.860    loss:   8.075
epoch:  6    weight 1:   2.666    weight 2:   3.152    loss:   6.577
epoch:  7    weight 1:   2.801    weight 2:   3.410    loss:   5.430
epoch:  8    weight 1:   2.907    weight 2:   3.639    loss:   4.530
epoch:  9    weight 1:   2.989    weight 2:   3.844    loss:   3.812
epoch: 10    weight 1:   3.053    weight 2:   4.030    loss:   3.229


### RMSprop
* Equation
$$
\begin{align}\mathbf{w}^{(t+1)}&=\mathbf{w}^{(t)}-\frac{\eta}{\sqrt{G^{(t)}} + \epsilon}\mathbf{g}^{(t)}\\
\mathbf{g}^{(t)}&=\nabla_\mathbf{w}L(\mathbf{w}^{(t)})\\
G^{(t)}&=\rho G^{(t-1)}+(1-\rho)\mathbf{g}^{(t)T}\mathbf{g}^{(t)}
\end{align}
$$
* Parameters
    * $\epsilon$ is a constant that make sure the denominator is not zero (default in Tensorflow/Keras $1e-7$).
    * $\rho$ is the parameter that control how much the model should consider the computed gradient (sum of magnitude) before. (default in Tensorflow/Keras $0.9$)
    * $\eta$ is the learning rate (default in Tensorflow/Keras $0.001$)

* Properties
    * Converge even with large learning rate.
    * Vunerable to local minimum.

In [4]:
eta = 1
w = tf.Variable(tf.zeros((2, 1)))
opt = optimizers.RMSprop(learning_rate=eta)
train(num_epochs, opt, square_loss, w)

epoch:  0    weight 1:   0.000    weight 2:   0.000    loss:  37.729
epoch:  1    weight 1:   3.162    weight 2:   3.162    loss:   6.049
epoch:  2    weight 1:   3.368    weight 2:   4.563    loss:   1.867
epoch:  3    weight 1:   3.282    weight 2:   5.347    loss:   0.576
epoch:  4    weight 1:   3.218    weight 2:   5.800    loss:   0.165
epoch:  5    weight 1:   3.182    weight 2:   6.055    loss:   0.043
epoch:  6    weight 1:   3.163    weight 2:   6.193    loss:   0.010
epoch:  7    weight 1:   3.153    weight 2:   6.263    loss:   0.002
epoch:  8    weight 1:   3.149    weight 2:   6.297    loss:   0.000
epoch:  9    weight 1:   3.146    weight 2:   6.313    loss:   0.000
epoch: 10    weight 1:   3.146    weight 2:   6.319    loss:   0.000


## Momemtum
* Equation
$$
\begin{align}\mathbf{w}^{(t+1)}&=\mathbf{w}^{(t)}-\mathbf{v}^{(t)}\\
\mathbf{v}^{(t)}&=\gamma\mathbf{v}^{(t-1)}+\eta\nabla_\mathbf{w}L(\mathbf{w}^{(t)})
\end{align}
$$

* Parameters
    * $\gamma$ is the momemteum parameter that control how much the model should consider the computed gradient before.

* Properties
    * Implemented in `SGD` and `RMSProp` in Tensorflow and Keras.
    * Can generalize well to most of the problems.
    * Consider both magnitude and direction of previous gradient when update the parameters.
    * Might not converge with large learning rate.
    * Less vunerable to local minimum.


In [5]:
eta = 0.1
w = tf.Variable(tf.zeros((2, 1)))
opt = optimizers.SGD(learning_rate=eta, momentum=0.1)
train(num_epochs, opt, square_loss, w)

epoch:  0    weight 1:   0.000    weight 2:   0.000    loss:  37.729
epoch:  1    weight 1:   0.757    weight 2:   0.817    loss:  26.427
epoch:  2    weight 1:   1.419    weight 2:   1.604    loss:  17.908
epoch:  3    weight 1:   1.920    weight 2:   2.283    loss:  12.251
epoch:  4    weight 1:   2.290    weight 2:   2.860    loss:   8.505
epoch:  5    weight 1:   2.561    weight 2:   3.351    loss:   5.987
epoch:  6    weight 1:   2.757    weight 2:   3.770    loss:   4.266
epoch:  7    weight 1:   2.898    weight 2:   4.128    loss:   3.072
epoch:  8    weight 1:   2.999    weight 2:   4.434    loss:   2.230
epoch:  9    weight 1:   3.069    weight 2:   4.696    loss:   1.631
epoch: 10    weight 1:   3.118    weight 2:   4.921    loss:   1.199


## Nesterov Accelerate Gradient (NAG)
* Equation
\begin{align}
\mathbf{w}^{(t+1)}&=\mathbf{v}^{(t+1)}-\eta\nabla_\mathbf{v}L(\mathbf{v}^{(t)})\\
\mathbf{v}^{(t+1)}&=\mathbf{w}^{(t)}+\gamma(\mathbf{w}^{(t)}-\mathbf{w}^{(t-1)})
\end{align}
* Parameters
    * $\gamma$ is the momemteum parameter that control how much the model should consider the computed gradient before.

* Properties
    * Implemented in `SGD` in Tensorflow and Keras.
    * Can generalize well to most of the problems.
    * Consider both magnitude and direction of previous gradient when update the parameters.
    * Gradients in earlier iterations become less informative, and therefore usually reach convergence faster than momemtum.
    * Might not converge with large learning rate.
    * Less vunerable to local minimum.

In [6]:
eta = 0.1
w = tf.Variable(tf.zeros((2, 1)))
opt = optimizers.SGD(learning_rate=eta, momentum=0.1, nesterov=True)
train(num_epochs, opt, square_loss, w)

epoch:  0    weight 1:   0.000    weight 2:   0.000    loss:  37.729
epoch:  1    weight 1:   0.833    weight 2:   0.899    loss:  25.418
epoch:  2    weight 1:   1.466    weight 2:   1.671    loss:  17.308
epoch:  3    weight 1:   1.939    weight 2:   2.329    loss:  11.958
epoch:  4    weight 1:   2.291    weight 2:   2.889    loss:   8.377
epoch:  5    weight 1:   2.551    weight 2:   3.368    loss:   5.943
epoch:  6    weight 1:   2.742    weight 2:   3.777    loss:   4.262
epoch:  7    weight 1:   2.881    weight 2:   4.128    loss:   3.086
epoch:  8    weight 1:   2.981    weight 2:   4.429    loss:   2.252
epoch:  9    weight 1:   3.053    weight 2:   4.688    loss:   1.654
epoch: 10    weight 1:   3.104    weight 2:   4.910    loss:   1.221


## Adam
* Equation
\begin{align}
\mathbf{w}^{(t+1)}&=\mathbf{w}^{(t)}-\frac{\eta}{\sqrt{\hat{v}^{(t)}} + \epsilon}\mathbf{\hat{m}}^{(t)}\\
\mathbf{g}^{(t)}&=\nabla_\mathbf{w}L(\mathbf{w}^{(t)})\\
G^{(t)}&=\mathbf{g}^{(t)T}\mathbf{g}^{(t)}\\
\mathbf{m}^{(t)}&=\beta_1\mathbf{m}^{(t-1)}+(1-\beta_1)\mathbf{g}^{(t)}\\
v^{(t)}&=\beta_2v^{(t-1)}+(1-\beta_2)G^{(t)}\\
\mathbf{\hat{m}}^{(t)}&=\frac{\mathbf{m^{(t)}}}{1-\beta_1^t}\\
v^{(t)}&=\frac{v^{(t)}}{1-\beta_2^t}\\
\end{align}
* Parameters
    * $\epsilon$ is a constant that make sure the denominator is not zero. (default in Tensorflow/Keras $1e-7$).
    * $\beta_1$ is the parameter that control how much the model should consider the computed gradient (consider both direction and magnitude) before. (default in Tensorflow/Keras $0.9$)
    * $\beta_2$ is the parameter that control how much the model should consider the computed gradient (sum of magnitude) before. (default in Tensorflow/Keras $0.999$)
    * $\eta$ is the learning rate (default in Tensorflow/Keras $0.001$)
* Properties
    * Consider both magnitude and direction of previous gradient when update the parameters.
    * Usually converge slower than momentum and NAG.
    * Converge even with large learning rate.
    * Less vunerable to local minimum.



In [7]:
eta = 1
w = tf.Variable(tf.zeros((2, 1)))
opt = optimizers.Adam(learning_rate=eta)
train(num_epochs, opt, square_loss, w)

epoch:  0    weight 1:   0.000    weight 2:   0.000    loss:  37.729
epoch:  1    weight 1:   1.000    weight 2:   1.000    loss:  23.800
epoch:  2    weight 1:   1.976    weight 2:   1.991    loss:  13.610
epoch:  3    weight 1:   2.896    weight 2:   2.964    loss:   7.039
epoch:  4    weight 1:   3.707    weight 2:   3.907    loss:   3.652
epoch:  5    weight 1:   4.347    weight 2:   4.805    loss:   2.615
epoch:  6    weight 1:   4.767    weight 2:   5.638    loss:   2.853
epoch:  7    weight 1:   4.960    weight 2:   6.387    loss:   3.457
epoch:  8    weight 1:   4.952    weight 2:   7.034    loss:   3.919
epoch:  9    weight 1:   4.787    weight 2:   7.562    loss:   4.068
epoch: 10    weight 1:   4.504    weight 2:   7.965    loss:   3.918


## Nesterov-accelerated Adam (Nadam)
* Equation
\begin{align}
w^{(t)}&=w^{(t-1)}-\frac{\eta}{\sqrt{\hat{v}^{(t)}} + \epsilon}(\beta_1\hat{m}^{(t)}+(1-\beta_1)\hat{g}^{(t)})\\
\mathbf{g}^{(t)}&=\nabla_\mathbf{w}L(\mathbf{w}^{(t)})\\
G^{(t)}&=\mathbf{g}^{(t)T}\mathbf{g}^{(t)}\\
\mathbf{m}^{(t)}&=\beta_1\mathbf{m}^{(t-1)}+(1-\beta_1)\mathbf{g}^{(t)}\\
v^{(t)}&=\beta_2v^{(t-1)}+(1-\beta_2)G^{(t)}\\
\mathbf{\hat{g}}^{(t)}&=\frac{\mathbf{g}^{(t)}}{1-\beta_1^t}\\
\mathbf{\hat{m}}^{(t)}&=\frac{\mathbf{m}^{(t)}}{1-\beta_1^{(t+1)}}\\
\hat{v}^{(t)}&=\frac{v^{(t)}}{1-\beta_2^t}\\
\end{align}
* Parameters
    * $\epsilon$ is a constant that make sure the denominator is not zero. (default in Tensorflow/Keras $1e-7$).
    * $\beta_1$ is the parameter that control how much the model should consider the computed gradient (direction and magnitude) before. (default in Tensorflow/Keras $0.9$)
    * $\beta_2$ is the parameter that control how much the model should consider the computed gradient (sum of magnitude) before. (default in Tensorflow/Keras $0.999$)
    * $\eta$ is the learning rate (default in Tensorflow/Keras $0.001$)
* Properties
    * Perform similarly to Adam, but usually converge faster.
    * Consider both magnitude and direction of previous gradient when update the parameters.
    * Converge even with large learning rate.
    * Less vunerable to local minimum.

In [8]:
eta = 1
w = tf.Variable(tf.zeros((2, 1)))
opt = optimizers.Nadam(learning_rate=eta)
train(num_epochs, opt, square_loss, w)

epoch:  0    weight 1:   0.000    weight 2:   0.000    loss:  37.729
epoch:  1    weight 1:   1.056    weight 2:   1.056    loss:  23.121
epoch:  2    weight 1:   1.700    weight 2:   1.769    loss:  15.800
epoch:  3    weight 1:   2.215    weight 2:   2.394    loss:  10.844
epoch:  4    weight 1:   2.643    weight 2:   2.979    loss:   7.311
epoch:  5    weight 1:   2.988    weight 2:   3.534    loss:   4.814
epoch:  6    weight 1:   3.247    weight 2:   4.056    loss:   3.091
epoch:  7    weight 1:   3.424    weight 2:   4.541    loss:   1.927
epoch:  8    weight 1:   3.525    weight 2:   4.985    loss:   1.156
epoch:  9    weight 1:   3.565    weight 2:   5.381    loss:   0.659
epoch: 10    weight 1:   3.558    weight 2:   5.727    loss:   0.355
